In [1]:
from docx import *
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
# read file
document = Document("Converted/just ind-eng-1-69.docx")
# document = Document("Converted/pdf2go-mini.docx")

In [3]:
for paragraph in document.paragraphs:
    
    # first line indent (segitiga atas di ruler)
    first = paragraph.paragraph_format.first_line_indent
    
    # left indent (segitiga bawah di ruler)
    left = paragraph.paragraph_format.left_indent
    
    print(first,left,paragraph.text+"\n")

None None A

None 0 

None 0 

None 0 

None 0 

None None 

-152400 227965 a and A I /a/ the ¤rst letter of the Latin alphabet used for writing Indonesian.

-152400 227965 a II (abbr) are (a unit of square measure in the metric system, equal to 100 square meters or about 119.6 square yards).

None 75565 a III (abbr) ampere.

None 75565 a IV ahh (hesitation).

-152400 227965 A V (in music) the sixth tone/note in the ascending scale of C ma<jor.

None 75565 A VI car license plate for Banten.

None 75565 à and @ at. – Rp 1.000,- at Rp 1,000.00.

None 75565 A ’45 [Angkatan ’45] the Generation of 1945.

None 75565 AA car license plate for Kedu (Magelang).

None 75565 aah ah. –, itu kan proyék swasta. Ah, that’s a private project.

None 75565 aala and a’ala dynasty;  RAJAKULA. – Tang Tang dynasty.

None 75565 aam (A) general. Rois – General Chairman (of the NU);  AM.

None 75565 aau ow! ouch!

None 75565 ab I (A ob) father.

None 75565 ab II (ob) a tin opium box/cylinder. AB III car licen

0 None lama pusaka usang (M) custom is unchanging. – gajah terdo- rong a ruler usually misuses his power. – diisi, lembaga dituang

None 229235 (M) habit/custom becomes second nature. – bersendi syarak, syarak bersendi –/kitabullah (M) no deed/action should be in con¶ict with adat and religion. – sepanjang jalan, cupak sepan- jang betung so many countries, so many customs. – periuk ber- kerak, – lesung berdekak no gain without pain. – berkabung funerary customs. – bersendi syara’ custom based on Islamic law. – cara folklore. – gelanggang regulations for cock¤ghting.

0 228600 (dan) istiadat customs and traditions. – yang kawi the adat handed down from the past. – kebiasaan habits, customs. – ke- manakan (M) customs concerning inheritance through the fe<male line. – lembaga customs and traditions. – pemaduan a penalty paid by a man when taking a second wife. – pemali pen<alty paid for having violated adat. – pembuang penalty paid in a divorce. – pertunangan betrothal customs. – resam cu

-635 228600 – beli purchasing agent. – dagang commercial/manufacturer’s agent. – duane custom’s agent. – ganda double agent. – jénderal general agent. – jual sales agent. – kepala head agent. – kuasa agent holding a power of attorney. – muda subagent. – pelaksana executing agency. – pembaru trend setter. – pembayar(an) pay<ing agent. – pembelian purchasing agent. – penagihan collection agent. – penidur sleeper agent. – penjual placement agent. – pen- jualan selling agent. – perjalanan travel agent. – perkapalan ship<ping agent. – polisi policeman. – rahasia secret agent. – rangkap double agent. – samaran undercover agent. – surat kabar local newspaper distributor. – teri minor agent. – tunggal sole agent/ representative. mengagén-tunggali to be the sole agent for. keagén-tunggalan and keagénan tunggal sole agency/repre<sentation. – umum general agent.

None 152400 mengagéni to be an agent for, represent.

None 152400 keagénan and peragénan agency, representation.

None 152400 pengagéna


0 None benang ¤liform root. – besar blue sky ¶ower/vine, blue trum<pet vine, Thunbergia grandi¶ora. – biji k.o. plant, Roucheria grif¤thiana. – binasa scarlet leadwort, Plumbago indica. – bulu hair root. – (ber)gantung aerial root. – bukit various species of plants, Anonaceae. – bumi k.o. plant, Rennellia paniculata. – cina k.o. liana, Limacia oblonga. – cabang lateral root. – dagun

None 227965 k.o. plant, Gnetum tenuifolium. – egrang tap root. – gamat k.o. plant, Pericampylus glaucus. – gambir a climber, jungle weed, Combretum sundaicum. – gantung aerial root. – gigi root of a tooth. – isap sucker, haustorium. – jangkar anchor root. – jitah

None 227965 k.o. liana. – kambing-kambing a climber, Sarcolobus globosus. – kemenyan k.o. climber, Chinese yam, Dioscorea oppositifolia. – kuayah  KUAYAH. – kubik cube root. – kucing k.o. climber, Toddalia aculeata. – kuning/ kunyit k.o. climber used for me<dicinal purposes, Fibraurea chloroleuca. – kupur k.o. plant, Cey<lon blackberry, Rubus m

0 227965 cacah counter. –.– cerna digestive organs. – cukur shaver. – dalam viscera. – dapur kitchen utensils. – détéksi a) detector. b) “sniffer” (at airport). – éléktronik electronic device. – gali exca<vator. – gempa seismograph. – gésék a) string instrument. b) de<vice for taking an imprint of credit cards. – getar vibrator. – hias toilet article. – hias tutup dinding wainscoting. – hisap drug para<phernalia for smoking narcotics. –.– hubungan semesta mass communications media. – hukum legal remedy. – indria sense organ. – kandungan womb. –.– kantor of¤ce equipment/sup<plies. – keamanan security organ. – keamanan untuk memeriksa penumpang metal detector (in airports). – kebesaran insignia of State. –.– kecantikan cosmetics. –.– kedokteran medical equip<ment. – kekuasaan machinery of power. – kelamin sex organ.

None 227965 –.– kelengkapan a) State organs, organs of State. b) parapherna<lia. – kemudi steering gear. –.– keperluan necessities. – kerajaan insignia of royalty, regalia. 

...

-153035 228600 antara I (Skr) 1 space between, distance between. jarak – X dan Y the distance between X and Y. – ada dengan tiada hardly, barely. – lain among others, including, inter alia. di – between, among. 2 in about (of time). – lima hari in about ¤ve days. tak lama – itu not long after that/thereafter. pada – itu at that very moment. 3 not far from. 4 in the meantime. 5 interim (deci<sion/position), intermediate. – kepala staf interim chief of staff.

-76200 228600 antaranya 1 among them. di ~ among others. 2 later. tiga hari ~ three days later. 3 distance between. ~ dari … ke … the dis<tance between ... and Y.

None 152400 seantara all around, entirely.

-76200 228600 berantara and berantara-antara 1 at intervals. 2 to be at a dis<tance. 3 to have an intermediary. berantarakan to be separated from e.o. by.

None 152400 
memerantarai (from the stem perantara) and memperantarai

None None to act as mediator for, mediate.

None 152400 mengantara  MENGANTARAI 2.

None 152400 menga

-152400 229235 apik I 1 (Jv) smart, neat, nice, attractive, chic, tidy, orderly. pakaian/ busana – chic clothes. 2 (J) very good (regarding maintenance). dengan – with care, very well.

-76200 229235 mengapikkan 1 to straighten up. 2 (ob) to care about; to pay at<tention to.

None 153035 keapikan neatness, tidiness.

None 153035 apik-apikan nicely.

None 76835 apik II (BG) [agak pikun] pretty senile.

None 76835 apikal (D ling) apical.

None 76835 apikultur (D) apiculture.

-152400 229235 apilan (cla) parapet, gun shield, rampart, bulwark, wooden breast<work before guns at mast of Malay pirate ship.

None None 

None 76200 apion	52

None None 

None 0 

None 152400 berapilan with this rampart, etc.

None 76200 apion (ob) spinning top.

-152400 227965 apit 1 to be wedged/hemmed in between two surfaces, be between two things/persons, etc. bulan – the eleventh Muslim month of Zulkaedah (wedged between Ramadan and Zulhijjah). 2  BERAPIT. 3 s.t. used as a wedge, clip, clamp. – lempang (nau

-76200 227965 ngarti (J) to understand, comprehend. sudah ~ tamed, domes<ticated (of a bird, etc.).

-76200 227965 mengartikan [and ngartiin (J coq)] 1 to interpret, set forth the meaning of, explain, elucidate. 2 to lead one to understand, help to understand.

None 151765 terartikan (infr) can be interpreted.

-76200 227965 artian meaning, sense, interpretation. jabatan rangkap dalam ~ kedinasan holding two jobs at the same time, in the sense of service. dalam ~ luas in a broad sense.

None 151765 arti-artian various k.o. meanings.

-76200 227965 pengartian understanding, comprehension, grasp, apprehension, perception.

None 75565 artifak (D) artifact.

None 75565 arti¤sial (D) arti¤cial.

76200 75565 kearti¤sialan arti¤ciality. arti¤sialitas (D) arti¤ciality. artik(a) (D) arctic (ocean).

None None artikel (D/E) 1 article (in newspaper). 2 clause. 3 (gram) article.

None None – karét (leg) a clause subject to various interpretations. – tentu

None 227965 (gram) de¤nite article. – tak


None 76200 astagatra (Skr neo) eight aspects of national life.

None 75565 astagina (Skr) handicraft, manual training (in school).

-153035 228600 astaka balai – (cla) hall without walls in a place where ceremonies are held.

None 76200 astakona (Skr) 1 octagon. 2 polygon.

76200 76200 berastakona octagonal. astana  ISTANA. astasatah octahedron. astasudut octagon.

None 76200 asték [asuransi sosial tenaga kerja] workers’ social insurance.

None 76200 Asteng [Asia Tenggara] Southeast Asia.

None 76200 asténia (D) asthenia.

None 76200 Astér [Asistén Téritorial] Assistant for Territorial Affairs.

None 76200 asteroid (D) asteroid. astergliserol (D) asterglycerol. astohpirulah  ASTAGAFIRULLAH.

None 76200 astra (Bal) insulting term for child born out of wedlock. astrada [asistén sutradara] assistant movie/play director. astral (D/E) astral.

None 76200 astraléla  ASTAGA.

None 76200 astringén (D/E) astringent.

None 76200 astro¤sik(a) (D) astrophysics. ahli – astrophysicist.

None 762

None 151765 pengawét 1 preservative. ~ kayu wood preservative. 2

None 227965 preserver.

-76200 227965 pengawétan 1 conservation. ~ susu pasteurization. 2 preserva<tion. ~ tanah soil conservation.

None 75565 awéwé (S) 1 woman. 2 wife.

None 76200 awi (S) various species of bamboo;  AUR I, BAMBU, BULUH. – andong

None 228600 k.o. cream­striped bamboo, Gigantochloa verticillata/pseudoa- rundinacea. – buluh yellow­grove bamboo, Schizostachyum brachycladum. – bunar k.o. bamboo, Schizostachyum blumei. – gereng spiny bamboo, Bambusa spinosa. – gombong k.o. bamboo cultivated for building materials, Gigantochloa pseudoarundi- nacea. – krisik hedge bamboo, Bambusa multiplex. – lengka blackhair giant grass, Gigantochloa nigrociliata. – tali k.o. bam<boo, Gigantochloa apus.

None 76200 awig-awig (Bal) adat regulations.

-152400 228600 awignam astu (namas sidem) (Skr) may it come true! may there be no obstacle to it!

-635 76200 awin  PENGAWINAN. awing (Pr) thief. awit I (ob)  AWÉT.

None 762

In [4]:
# Target yang di crawling
# [lema, sublema, gabungan(-,~), similar(panah,"and"), keterangan, akronim([]), terjemahan]

# CARA NGEDAPETIN (bantu isi cara ngedapetin atau ciri khas atau langkah2 buat ngedapetin datanya)
# [v] == sudah dibuat codingannya

# [v] LEMA : indent pertama,bold (simpan di tempLema)
# [v] LEMA (jika sebaris dg lema lainnya) : indent pertama, jika kondisi tempLema isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempLema
# [v] LEMA (jika sebaris dg sublema) : indent kedua, jika kondisi tempSublema harus isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempSublema

# [v] SUBLEMA : indent kedua,deteksi bold (simpan di tempSublema)
# [v] SUBLEMA (jika sebaris dg lema): indent pertama, jika kondisi tempLema isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempLema
# [v] SUBLEMA (jika sebaris dg sublema lainnya) : indent kedua, jika kondisi tempSublema harus isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempSublema

# [v] GABUNGAN(-,~) : deteksi kata italic, ambil kata-katanya simpan ke list tempGabungan, cek di list ada - atau ~ tidak, kalau ada berarti itu kata gabungan dan replace - dan ~ dengan lema/sublema 

# [v] SIMILAR ("and") : indent pertama/kedua, deteksi bold, jika kata sebelumnya adalah and maka similar
# SIMILAR (panah) :

# [v] KETERANGAN :

# [v] AKRONIM :

# TERJEMAHAN :

In [5]:
# Perisiapan Word :
# - ubah panah jadi >
# - –/~ yang reguler dan bold dijadikan italic
# - ubah font, kemudian tanda yang tidak terbaca seperti di kata "major" (-^w) diganti menjadi <
# - tanda yang tidak terbaca seperti di kata "brother-in-law" (-) diganti menjadi ^w
# - tanda "- " diganti menjadi "<", contoh me- ngelabui (PERTIMBANGKAN LAGI)
# - ubah tanda "..." menjadi "_" (garis bawah)
# - s.t. = something
# - s.o.'s = someone's
# - s.o. = someone
# - q.v. = which see (refer to that word)
# - k.o. = kind(s) of
# - o.s. = oneself
# - e.o. = eachother
# - replace .^w jadi .^p
# - menghapus setiap header halaman dan garisnya



In [6]:
re.sub("\s\w\)$",""," agag a) g)")

' agag a)'

In [7]:
def cleanWord(word) :
    #cek spasi pada kata (jika terdiri dari 2 kata)
    if re.search("\s", word) :
        word = word.split()

        word[:] = [w for w in word if not (w in roman_numerals or re.search("\d", w) or word == "/" or w == "." or w == "(")]
        
        word = " ".join(word)
        
        if len(word) < 1 :
            word = None

    #jika terdiri dari 1 kata
    else :
        if word in roman_numerals or re.search("\d", word) or word == "/" or word == "." or word == "(":
            word = None
    
    return(word)

def replaceData (x) :
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*","",x[idx])
        x[idx] = re.sub("^\)|^\)\s*","",x[idx])
        x[idx] = re.sub("\(\s\)","",x[idx])
        x[idx] = re.sub("^\w\)\s*|\s\w\)$","",x[idx])
        x[idx] = re.sub("^\s*|^\s|^\s\w","",x[idx])
        x[idx] = re.sub("\[and","",x[idx])
        x[idx] = re.sub("\[|\[\s","",x[idx])
        x[idx] = re.sub("\]","",x[idx])
        x[idx] = re.sub("\s-\s|-\s","-",x[idx])
        x[idx] = re.sub("\s<\s|\s<|<\s|<","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("\s/.*/\s","",x[idx])
        x[idx] = re.sub("/\s","/",x[idx])
#         x[idx] = re.sub("s\.t\.","something",x[idx])
#         x[idx] = re.sub("s\.o\.'s","someone's",x[idx])
#         x[idx] = re.sub("s\.o\.","someone",x[idx])
#         x[idx] = re.sub("q\.v\.","which see (refer to that word)",x[idx])
#         x[idx] = re.sub("k\.o\.","kind(s) of",x[idx])
#         x[idx] = re.sub("o\.s\.","oneself",x[idx])
#         x[idx] = re.sub("e\.o\.","eachother",x[idx])
#         x[idx] = re.sub(";|;\s","",x[idx])
        x[idx] = re.sub("\*\*","e.g.",x[idx])
        x[idx] = re.sub("\*","i.e.",x[idx])
        x[idx] = re.sub("\.$|\s\.$|\.\s$|\.\s","",x[idx])
#         x[idx] = re.sub("_","...",x[idx])
    
    return x

# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan):
    x = [lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan]
    
    x = replaceData(x)
    
    #split panah
    if x[4] != "" :
        x[4] = re.split(",|;",x[4])
    
    #split terjemahan
    if x[8] != "" :
        x[8] = re.split(",|;",x[8])
        x[8] = replaceData(x[8])
        
        if "" in x[8] :
            x[8].remove("")
        
        for idx in range(len(x[8])):
            x[8][idx] = re.sub("_","...",x[8][idx])
            
        if len(x[8]) < 1 :
            x[8] = ""
    
    y = pd.DataFrame(data=[x],columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","keterangan","terjemahan"])
    
    return x,y

def cleanParenthesis(word) :
    if '(' in word :
        x, y = re.sub(r"[/(].*?[/))=]","",word),re.sub(r"[()=]","",word)
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        word = x
        similar = y
        tempLemaSublema = word
        boolLemaSublema = True
    else :
        word = re.sub(r"[()]|\s$","",word)
        similar = ""
        tempLemaSublema = word
        boolLemaSublema = True
        
    return word,similar,tempLemaSublema,boolLemaSublema

def readItalicData(word,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan) :     
    if word == "opp" :
        word = ""

    #KETERANGAN
    # mengambil keterangan
    if index > 0 and re.search("\(", paragraph.runs[index-1].text):

        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)
            
        else :
            if paragraph.runs[index+1].italic :
                tempGabungan.append(word)
            else :
                if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = word
                elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = terjemahan+" "+word

        if re.search("\)", word) or re.search("\)", paragraph.runs[index+1].text) or re.search("\)", word) :
            keterangan = " ".join(tempKeterangan)
            tempKeterangan = []
        
#         keterangan = "satu"

    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
    elif len(tempKeterangan) > 0 and word not in tempKeterangan:
        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

            try :
                if re.search("\)", paragraph.runs[index+1].text) or re.search("\)", word) :
                    keterangan = " ".join(tempKeterangan)
                    tempKeterangan = []
            except :
                print("terjadi kesalahan")
                
        else :
            if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                terjemahan = word
            elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                terjemahan = terjemahan+" "+word
#         keterangan = "dua"

    else :
        tempGabungan.append(word)
#         keterangan = "tiga"
    
    return keterangan,tempKeterangan,tempGabungan,terjemahan

In [8]:
lexicon = pd.DataFrame(columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","keterangan","terjemahan"])
roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
list_and = ["and", " and", "and ", " and ", "[and", "[and "]
list_ket = ["A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","esp","euph","exclam","fin","Fr","G","geo","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","NTB","NTT","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","usu","voc","vulg","zod"]

#variabel
lema = ""
sublema = ""
gabungan = ""
similar = ""
panah = ""
akronim = ""
antonim = ""
keterangan = ""
terjemahan = ""

boolLemaSublema = False
boolPanah = False
tempLema = ""
tempSublema = ""
# tempLemaSublema = ""
tempTerjemahan = ""
tempGabungan = []
tempKeterangan = []
tempAkronim = []
    
index = 0

#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    boolLemaSublema = False
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #looping setiap kata di setiap paragrafnya
    for run in paragraph.runs:
        
#         print("word = "+run.text)
#         print("index = "+str(index))
#         print("length = "+str(len(paragraph.runs)))
        
        #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
        if left == None :
            left = 228600 
        
        #deteksi kata bold
        if run.bold and run.text.isspace() == False :
            if run.text.isspace() == False :
                #baca indent lema
                if (((first == None or first == 0 or first == 76200) and (left <= 76835)) or ((first == -152400 or first == -153035) and left >= 227965)) :      

                    tempLeft = left

                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                        print(x[0])
                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                    #variabel 
                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    keterangan = ""
                    terjemahan = ""

                    boolPanah = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []

                    word = cleanWord(run.text)

                    if word != None :
                        
#                         print("run.text = "+run.text)
#                         print(word)
                        
                        #LEMA
                        #jika lema kosong atau belum terdapat lema, lema = word
                        if boolLemaSublema == False :

                            results = cleanParenthesis(word)

                            lema = results[0]
                            similar = results[1]
#                             tempLemaSublema = results[2]
                            boolLemaSublema = [3]
                            
                            tempLema = lema

                        #jika lema tidak kosong atau variabel lema sudah terisi
                        else :
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SIMILAR (AND)
                            #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                            if paragraph.runs[index-1].text in list_and or paragraph.runs[index-2].text in list_and :
#                                 print(word)
                                similar = word

                            #jika bukan similar
                            else :  
                                #SUBLEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA)  
                                #jika sublema kosong
                                if sublema == "" :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                                #jika sublema tidak kosong, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    similar = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                #baca indent sublema
                else :
                    tempLeft = left 

                    #deteksi kata bold
                    if(run.bold) and run.text.isspace() == False :

                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        keterangan = ""
                        terjemahan = ""

                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                        word = cleanWord(run.text)

                        if word != None :
                            
#                             print("prev = "+paragraph.runs[index-1].text)
#                             print("word = "+word)
                            
                            #SUBLEMA
                            #jika sublema kosong
                            if boolLemaSublema == False :

                                results = cleanParenthesis(word)

                                sublema = results[0]
                                similar = results[1]
#                                 tempLemaSublema = results[2]
                                tempSublema = results[2]
                                boolLemaSublema = [3]

                            #jika sublema tidak kosong
                            else :
                                #SIMILAR (AND)
                                #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                                if paragraph.runs[index-1].text in list_and or paragraph.runs[index-2].text in list_and :
                                    similar = word

                                #SUBLEMA (JIKA SUBLEMA SEBARIS/SEPARAGRAF DENGAN SUBLEMA YANG LAIN)
                                #jika bukan, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    lema = ""
                                    sublema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    keterangan = ""
                                    terjemahan = ""

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
#                                     tempLemaSublema = results[2]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

        #deteksi kata italic (kata gabungan, keterangan)
#             print("sublema = "+sublema)
#             print("similar = "+similar)
#             print("tempLemaSublema = "+tempLemaSublema)
            
        elif run.italic :
            if run.text.isspace() == False :
                results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

#                 print("prev = "+paragraph.runs[index-1].text)
#                 print(paragraph.runs[index-1].italic)
#                 print("text italic = "+run.text)
                
#                 print(results)
                
                keterangan = results[0]
                tempKeterangan = results[1]
                tempGabungan = results[2] 
                terjemahan = results[3]
                
        # deteksi kata reguler (akronim, terjemahan)
        else : 
            if run.text.isspace() == False :
                word = run.text

                if word != None :
                    #GABUNGAN
#                     if "–" in tempGabungan or "~" in tempGabungan:
#                         gabungan = " ".join(tempGabungan)
#                         gabungan = re.sub("–|~",tempLemaSublema,gabungan)
#                         gabungan = re.sub("  "," ", gabungan)

#                         results = cleanParenthesis(gabungan)

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

                    if "–" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("–",tempLema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        gabungan = results[0]
                        similar = results[1]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []
                    
                    elif "~" in tempGabungan:
                        gabungan = " ".join(tempGabungan)
                        gabungan = re.sub("~",tempSublema,gabungan)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
                        gabungan = results[0]
                        similar = results[1]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)

                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

#                         gabungan = results[0]
#                         similar = results[1]

#                         tempGabungan = []

                    else :
                        if len(tempGabungan) != 0:
                            
#                             print ("length = "+str(len(tempGabungan)))
#                             print ("index = "+str(index))
#                             print(tempGabungan)
#                             print("tempLema = "+tempLema )

                            temp = " ".join(tempGabungan)
                            
                            if boolPanah == True and panah != word and word not in roman_numerals :
#                                 temp = " ".join(tempGabungan)
                                panah = panah+" "+temp

                                temp = ""
                                tempGabungan = []
                            
#                             elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempLemaSublema.lower() in temp.lower()) :
                            elif len(tempGabungan) > 1 and (tempLema.lower() in temp.lower() or tempSublema.lower() in temp.lower()) :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                gabungan = results[0]
                                similar = results[1]

                                tempGabungan = []
                            
                            elif index > 1 and len(tempGabungan) <= 3:
#                                 temp = " ".join(tempGabungan)
                                terjemahan = terjemahan+" "+temp

                                temp = ""
                                tempGabungan = []

                            else :
#                                 temp = " ".join(tempGabungan)
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

                                gabungan = results[0]
                                similar = results[1]

                                tempGabungan = []

                #AKRONIM
                # mengambil akronim
                if re.search("\[", paragraph.runs[index-1].text) and word not in list_and:
                    tempAkronim.append(word)

                    if re.search("\]", paragraph.runs[index+1].text) :
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                elif len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

                    try :
                        if re.search("\]", paragraph.runs[index+1].text) :
                            akronim = " ".join(tempAkronim)
                            akronim = re.sub(r'  ',' ', akronim)
                            tempAkronim = []
                    except :
                        if re.search("\]", paragraph.runs[index].text) :
                            akronim = " ".join(tempAkronim)
                            akronim = re.sub(r'  ',' ', akronim)
                            tempAkronim = []
                else :
                    #PANAH   
                    
#                     print("prev-1 = "+paragraph.runs[index-1].text)
                    
#                     if index > 1 :
#                         print("prev-2 = "+paragraph.runs[index-2].text)
                        
#                     print("word = "+word)
#                     print("terjemahan = "+terjemahan)
#                     print("index = "+str(index))
#                     print("length = "+str(len(paragraph.runs)))
                    
                    if boolPanah == True and word not in roman_numerals :
                        tempWord = re.sub("\.","", word)
                        tempWord = cleanWord(tempWord)

                        if tempWord != None :
                            panah = panah+" "+tempWord
                            word = ""
                            
#                         print("panah 2 = "+panah)
                    
                    if boolPanah == True and re.search("\.", word) :
                        panah = re.sub("\.|\s\.","",panah)
                        boolPanah = False
                        word = ""
#                         print("panah 3 = "+panah)
                    
                    
                    if word != "" and word != "." and (paragraph.runs[index-1].text == ">" or (index > 1 and paragraph.runs[index-2].text == ">")) :
#                         print("run.text = "+run.text)

                        word = re.sub("\.","", word)
                        word = cleanWord(word)

                        if word != None :
                            panah = word
                            boolPanah = True
                            word = ""
                            
#                         print("panah = "+panah)
                        
#                         print("panah 1 = "+panah)
                            
#                             try :
#                                 print(paragraph.runs[index+1].text)
#                                 print(panah)
#                             except :
#                                 print(paragraph.runs[index+0].text)
#                                 print(panah)

                    elif paragraph.runs[index-1].italic == True and (paragraph.runs[index-1].text == "opp" or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)

                        antonim = word

                    else :

                        #TERJEMAHAN
                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if word in list_and and (paragraph.runs[index-1].bold == True and (paragraph.runs[index+1].bold == True or paragraph.runs[index+2].bold == True)):
                            word = ""

                        if word == "[" or word == "]" or word == ">":
                            word = ""

#                         if terjemahan == "" and word not in roman_numerals and boolPanah != True :
                        if terjemahan == "" and boolPanah != True :
                            terjemahan = word
#                         elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                        elif terjemahan != "" and boolPanah != True:
                            terjemahan = terjemahan+" "+word

#                         print("prev = "+paragraph.runs[index-1].text)
#                         print("word = "+word)
#                         print("terjemahan = "+terjemahan)
#                         print("run.text = "+run.text)
                        if re.search("\.\s|\;|\!|\?", run.text) or re.match("\w\)", run.text) :
        
#                             print("hihi")
            
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)

                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            boolPanah = False
                            tempGabungan = []
                            tempKeterangan = []
                            tempAkronim = []
                            
#                             boolPanah = False
        index += 1                 

['', '', '', '', '', '', '', '', ['A']]
['a', '', '', '', '', '', '', '', '']
['', '', '', 'A', '', '', '', '', '']
['', '', '', '', '', '', '', '', ['a/the first letter of the Latin alphabet used for writing Indonesian']]
['a', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', 'abbr', ['are (a unit of square measure in the metric system', 'equal to 100 square meters or about 119.6 square yards)']]
['a', '', '', '', '', '', '', 'abbr', ['ampere']]
['a', '', '', '', '', '', '', '', ['ahh (hesitation)']]
['A', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', ['(in music) the sixth tone/note in the ascending scale of C major']]
['A', '', '', '', '', '', '', '', ['car license plate for Banten']]
['à', '', '', '', '', '', '', '', '']
['', '', '', '@', '', '', '', '', ['at–']]
['', '', '', '', '', '', '', '', ['Rp 1 000', '-at Rp 1', '000.00']]
['A', '', '', '', '', 'Angkatan ’45', '', '', ['the Generation of 1945']]
['AA', '', '', '', '', '', '', '', ['car license

['', 'mengabarkan', '', '', '', '', '', '', ['to reduce', 'lower']]
['', 'terabar', '', '', '', '', '', '', ['hindered', 'obstructed', 'impeded', 'etc']]
['', 'abaran', '', '', '', '', '', '', ['(psychological) inhibition']]
['', '', '', '', '', '', '', '', ['~ renjana emotional inhibition']]
['pengabaran', '', '', '', '', '', '', '', '']
['', '', '', 'perabaran', '', '', '', '', ['screening', 'sheltering', 'braking']]
['', 'Abasa', '', '', '', '', '', 'A', ['“He Frowned”']]
['', '', '', '', '', '', '', '', ['title of the 80th chapter of the Koran']]
['', 'Abassiyah', 'dinasti pengabaran', '', '', '', '', 'A', ['Abbasid']]
['', '', '', '', '', '', '', '', ['Abbasid dynasty']]
['abasti', '', '', '', '', 'alat pembasmi tikus', '', '', ['rodent exterminator']]
['abat', '', '', '', '', '', '', '', ['\uf0ae ABAD']]
['abat', '', '', '', '', '', '', '', ['kind(s) of fish', 'bream']]
['', '', '', '', '', '', '', '', ['Argyros spinifer']]
['abaté', '', 'serbuk/obat abaté', '', '', '', '', '', [

['', 'mengablak', '', '', '', '', '', '', '']
['', '', 'Mulutnya ~', '', '', '', '', 'coq', ['to be wide open (of the mouth)His mouth was wide open']]
['', 'mengablakkan', '', '', '', '', '', '', '']
['', '', '', 'ngablakin', '', '', '', 'J coq', '']
['', '', '', '', '', '', '', '', ['to open something wide']]
['', '', '', '', '', '', '', '', ['to disclose', 'reveal']]
['', '', '', '', '', '', '', '', ['to tell (a story)']]
['ablasi', '', '', '', '', '', '', 'D', ['ablation']]
['mengablasi', '', '', '', '', '', '', '', ['to ablate']]
['', 'ablatif', '', '', '', '', '', 'D/E', ['ablative']]
['', 'ablaut', '', '', '', '', '', 'D', ['(/E) ablaut']]
['', 'ablur', '', '', '', '', '', '', ['\uf0ae HABLUR']]
['abnormal', '', '', '', '', '', '', 'D/E', ['abnormal']]
['', 'keabnormalan', '', '', '', '', '', '', ['abnormality']]
['', '', '', '', '', '', '', '', ['\uf0ae ABNORMALITAS']]
['abnormalitas', '', '', '', '', '', '', 'D', ['abnormality']]
['', 'abnus', '', '', '', '', '', 'A cla', ['ebo

['', '', '', '', '', '', '', '', ['to stir up a hornet’s nest']]
['', '', '', '', '', '', '', '', ['– bara cinder']]
['', '', '', '', '', '', '', '', ['– batu bara coal ash']]
['', '', '', '', '', '', '', 'Jv', ['– blarak dried coconut leaf powder (used as a cleanser)– dasar bottom ash/slag– gosok scouring sand– kayu wood ash– limbah fly ash– soda sodium carbonate– terbang fly ash– vulkanik/vul-kanis volcanic ash']]
['', 'abu-abu', 'abu-abu muda', '', '', '', '', '', ['ashgray', 'ashyudara ~ an overcast sky']]
['', '', '', '', '', '', '', '', ['light gray']]
['', 'berabu', '', '', '', '', '', '', ['dusty']]
['', 'mengabu', '', '', '', '', '', '', ['to become ashes']]
['', 'mengabui', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', ['to spray/scatter ashes on something (in planting)']]
['', '', '', '', '', '', '', '', ['(=']]
['', 'me-', '', '', '', '', '', '', '']
['', 'ngelabui', '', '', '', '', '', '', ['to fool', 'trick~ mata to pull the wool over someone’s eyes', 'to d

['', '', '', '', '', '', '', '', ['agendatertib – rules of order']]
['', '', 'pengarah acara', '', '', '', '', '', ['subject', 'topic', 'theme', 'program', 'show']]
['', '', 'dalam acara rapat', '', '', '', '', '', ['Master of Ceremonies', 'emcee']]
['', '', '', '', '', '', '', '', ['on the agenda']]
['', '', '', '', '', '', '', '', ['lawsuit', 'court case']]
['', '', 'berita acara', '', '', '', '', '', ['judicial procedure']]
['', '', 'gedung acara', '', '', '', '', '', ['official report', 'deposition']]
['', '', 'hari acara', '', '', '', '', 'ob', ['court (of justice)', 'law courts']]
['', '', 'hukum acara', '', '', '', '', 'ob', ['court session']]
['', '', '', '', '', '', '', '', ['law of procedure']]
['', '', '', '', '', '', '', '', ['heading (of an article)']]
['', '', '', '', '', '', '', 'ob', ['method', 'way', 'manner']]
['', '', '', '', '', '', '', '', ['\uf0ae CARA – banding(an) appeal– bébas an unscheduled activity– bedah launch', 'opening– berperkara legal proceedings', 'lit

...

['', '', '', '', '', '', '', '', ['zat ~ catalyst']]
['pengaktifan', '', '', '', '', '', '', '', ['activation']]
['', 'aktifis', '', '', '', '', '', '', ['\uf0ae AKTIVIS']]
['', 'aktifitas', '', '', '', '', '', 'D', ['activity']]
['akting', '', 'ketua', '', '', '', '', 'E', ['temporarily taking over the duties of someone else', 'acting– acting chairman']]
['', 'berakting', '', '', '', '', '', '', ['to act', 'perform on the stage']]
['aktiva', '', '', '', '', '', '', 'D', ['assets']]
['', '', '', '', '', '', '', '', ['cp PASIVA – beku frozen assets– berakumulasi accrued assets– bergerak movable assets– berisiko risk assets– berwujud tangible assets– dijaminkan pledged assets– (yang) ditangguhkan deferred assets– jaminan pledged assets– lan-car current/liquid assets– menghasilkan earning assets– modal capital assets– nyata tangible assets– rupa-rupa miscellaneous assets– tak berwujud intangible assets– tetap fixed assets']]
['aktivis', '', '', '', '', '', '', 'D/E', ['(student) activist 

['akumulator', '', '', '', '', '', '', 'D/E', ['accumulator–']]
['', '', '', '', '', '', '', '', ['kalor heat accumulator']]
['akun', '', '', '', '', '', '', 'D', ['account']]
['', '', '', '', '', '', '', '', ['\uf0ae AKAUN – (ber)jaminan secured account– gabungan consolidated accounts– laba rugi profit and loss account']]
['', '', '', '', '', '', '', '', ['\uf0ae PERHITUNGAN laba-rugi –']]
['', '', '', '', '', '', '', '', ['lawan contra account– peruntukan appropriation account–']]
['', '', '', '', '', '', '', '', ['pindahan continuing account– sementara suspense account– tak-aktif dormant account']]
['', 'perakunan', '', '', '', '', '', '', ['accounting']]
['', '', '', '', '', '', '', '', ['~ bébas public accounting']]
['akuntabilitas', '', '', '', '', '', '', '', ['accountability']]
['akuntan', '', '', '', '', '', '', 'D/E', ['accountant– bertanggung jawab/penanggung jawab accountant in charge– publik public accountant– Publik Beri-jazah/Terdaftar Certified Public Accountant', 'CPA'

['', 'mengalahi', '', '', '', '', '', '', ['to admit defeat']]
['', '', '', '', '', '', '', 'ob', ['to defeat', 'subject']]
['', 'alah-mengalahi', '', '', '', '', '', '', ['to make concessions/give in to eachother']]
['', 'mengalahkan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', ['to defeat', 'subject', 'subdue', 'overwhelm']]
['', '', '', '', '', '', '', '', ['to declare someone defeated', 'declare someone the loser']]
['', '', '', '', '', '', '', '', ['to surpass', 'exceed', 'outdo']]
['', '', '', '', '', '', '', '', ['to oust', 'supersede (another language)dialahkan to be overgrown with (weeds/benalu)']]
['', 'teralahkan', 'tidak teralahkan', '', '', '', '', '', ['defeatable', 'can be beaten']]
['', '', '', '', '', '', '', '', ['invincible']]
['', 'alahan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', ['the loser']]
['', '', '', '', '', '', '', '', ['always losing']]
['', '', '', '', '', '', '', '', ['loss (in gambling)']]
['', 'kealahan', '', '', '

['', '', '', '', '', '', '', '', ['to indicate', 'foretell']]
['', 'teralamatkan', '', '', '', '', '', '', ['can be addressed', '(blame) can be laid at the feet of']]
['', 'pengalamatan', '', '', '', '', '', '', ['addressing']]
['', 'peralamatan', '', '', '', '', '', '', ['addressing']]
['', '', '', '', '', '', '', '', ['sistém penamaan dan ~ komputer computer naming and addressing system']]
['alamatulhayat', '', '', '', '', '', '', 'A', ['signs of life']]
['Alambhana', '', '', '', '', '', '', '', '']
['', 'Wana', '', '', '', '', '', '', '']
['', 'Wai', '', '', '', '', '', '', ['the military resort command (Korém 121) in Pontianak']]
['alami', '', 'musuh alami', '', '', '', '', 'A', ['natural']]
['', '', 'tak/tidak alami', '', '', '', '', '', ['natural enemy (of a pest)']]
['', '', 'secara alami', '', '', '', '', '', ['unnatural', 'artificial', 'manmade']]
['', '', '', '', '', '', '', '', ['in a natural way']]
['alamiah', '', '', '', '', '', '', 'A', '']
['', '', 'pecandu alamiah', '',

IndexError: list index out of range

In [ ]:
lexicon